**Summary**

Curve fitting can be approached in many different ways. This notebook compares my "standard" method---writing a function to return the sum of residual squares, and then minimizing it with `scipy.optimize.fmin`---with the python library `lmfit`, which in principle wraps around similar functions in the `scipy.optimize` library, and provides some extra functions, e.g. to constrain the optimized variables and to give confidence intervals. The example here comes from our recent attempts to use a model to predict protein adsorption to inline filters, based on an experiment.

The fitted parameters were notably quite different, although the curves in the end look roughly the same. This data set is probably not enough to fully compare the methods. The `lmfit` library provides some convenient functions and is fairly easy to implement, but unfortunately in this case setting parameter constraints caused the confidence interval function to fail.

**Import libraries and set plotting options:**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.stats import linregress
from scipy.optimize import fmin
import lmfit

plt.style.use('bmh')
%config InlineBackend.figure_format = 'svg'

%load_ext watermark

In [2]:
%watermark -v -m -p numpy,scipy,matplotlib,lmfit -g

CPython 3.6.4
IPython 6.2.1

numpy 1.14.0
scipy 1.0.0
matplotlib 2.1.2
lmfit 0.9.7

compiler   : GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 17.4.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit
Git hash   : 6d9317b5ef41553154c411e259d6fe530de06788


In [4]:
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__
x = list(imports())

**Raw data**

The data comes from an experiment monitoring the protein concentration after a single inline filter. The formulated protein has a nominal concentration of 5 mg/mL. The filter was wetted with the protein solution for a few minutes before the valve was opened to allow flow.

In [7]:
%watermark -v -m -p ['numpy', 'scipy']

UsageError: unrecognized arguments: 'scipy' ]


In [2]:
time = np.array([0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900, 930, 960, 990, 1020, 1050, 1080, 1110, 1140, 1170, 1200, 1230, 1260, 1290, 1320, 1350, 1380, 1410, 1440, 1470, 1500])

volume = np.array([0, 13.2522, 22.8151, 35.4172, 48.2249, 63.3103, 73.558, 86.1327, 96.7225, 107.9742, 118.6576, 128.7267, 140.6426, 152.4861, 162.8935, 174.1857, 185.7268, 197.1192, 209.5242, 221.4678, 232.2868, 247.6275, 259.4228, 270.9724, 281.8857, 293.4212, 304.6134, 315.6425, 326.8809, 338.671, 348.127, 363.4559, 374.8323, 385.8517, 396.7044, 408.1897, 420.0901, 427.0895, 437.5906, 448.138, 459.2603, 469.2068, 480.2171, 491.9888, 502.4742, 513.0803, 524.2078, 535.2854, 545.8359, 555.0863, 564.3464])

concentration = np.array([3.023798609, 3.114830065, 3.193132997, 3.344098032, 3.545952177, 3.71131258, 3.880179942, 4.051567078, 4.145738602, 4.204649925, 4.310641861, 4.373957515, 4.449874592, 4.523475265, 4.557658434, 4.621247387, 4.665646744, 4.688958049, 4.724696445, 4.763434982, 4.777761936, 4.817210579, 4.840685558, 4.846937061, 4.861563492, 4.862069321, 4.865633011, 4.883685875, 4.888158989, 4.893807769, 4.89773283, 4.902150154, 4.908982992, 4.91435442, 4.919867516, 4.916160679, 4.928572059, 4.917313385, 4.922553825, 4.919561267, 4.92966497, 4.934463024, 4.935445404, 4.937192059, 4.930878401, 4.932299042, 4.933749437, 4.929975319, 4.935555362, 4.937137485, 4.940053844])

**Additional constants**

$c$ denotes the concentration in the liquid phase, $q$ the concentration in the adsorbed phase.

Note that $q_{initial}$ comes from a mass balance from the initial equilibration period (no raw data).

In [3]:
Vol = 670 # mL
Vdot = linregress(time, volume).slope # mL/s
c_input = 4.94 # mg/mL
c_init = concentration[0] # ca. 3 mg/mL
q_init = c_input - c_init

In [1]:
#            __V___
#       Vdot|     |Vdot
#   c_in -->| c,q |--> c_out
#           |_____|
#           

**Part 1: Using `lmfit` to fit the single-filter data**

Concentration in liquid phase, $c$
$$\frac{dc}{dt} = \frac{\dot{V}}{V} (c_{input} - c) - k_1 c (q_{max} - q) + k_2 q$$

Concentration in adsorbed phase, $q$
$$\frac{dq}{dt} = k_1 c (q_{max} - q) - k_2 q$$

Note: at equilibrium $\frac{dq}{dt} = 0$ and $q = q_{max} \frac{(k_1/k_2) c_e}{1 + (k_1/k_2) c_e}$ (Langmuir isotherm)

Hashim, M. A.; Chu, K. H. Prediction of Protein Breakthrough Behavior Using Simplified Analytical Solutions. *Sep. Purif. Technol.* **2007**, 53 (2), 189–197 DOI: 10.1016/j.seppur.2006.06.028.

In [4]:
def rates(state, t, paras):
    c, q = state

    k1 = paras[0]
    k2 = paras[1]
    qmax = paras[2]
    V = paras[3]

    dcdt = Vdot / V * (c_input - c) - k1 * c * (qmax - q) + k2 * q
    dqdt = k1 * c * (qmax - q) - k2 * q
    return [dcdt, dqdt]


def conc(t, c0, q0, k1, k2, qmax, V):
    initial_state = (c0, q0)
    params_list = [k1, k2, qmax, V]
    x = odeint(rates, initial_state, t, args=(params_list, ))
    return x

**Additional functions**

I had to write this function, which will be used to create the `lmfit.Model` (a class), because the previously defined `conc` function returns both $c$ and $q$, while we only have data for $c$. When I tried to use `conc` directly for the Model, an error gets thrown during fitting because of the different dimensions of the output array and the data array. There is probably a cleaner way to address this.

In [5]:
def conc_index(t, c0, q0, k1, k2, qmax, index, V=Vol):
    x = conc(t, c0, q0, k1, k2, qmax, V)
    return x[:,index]

**Using the `lmfit.Model` class**

We make the object `c_model` by calling the `lmfit.Model` class, with the `conc_index` function passed as the model function, and the independent variable explicitly identified.

In the line that is commented out, I originally used the index as an independent variable (the index variable is only used to pick out the $c$ or $q$ values). This makes more logical sense; the only problem is that when you have more than one independent variable, you lose some of the convenient plotting methods of the `Model` class. So I faked it this way, I just have to be careful to make sure that the fit function does not try to change the index variable.

In [6]:
c_model = lmfit.Model(conc_index, independent_vars='t')

# c_model = lmfit.Model(conc_index, independent_vars=['t','index'])

**Setting Parameters**

The `make_params()` method will take the rest of the variables that were not marked as independent when making the `Model` object and turn them into `Parameter` objects, stored in the `params` object. Once they are made, we can set their initial values with the `value` attribute, and whether they vary or not. We can also set `min` and `max` attributes later.

For now we will fix $index$, $c_0$, $q_0$, and $V$, and allow $k_1$, $k_2$, and $q_{max}$ to vary.

In [7]:
params = c_model.make_params()

params['index'].value = 0 # This will return the c values
params['index'].vary = False

params['k1'].value = 0.01

params['k2'].value = 0.01

params['qmax'].value = 1

params['c0'].value = c_init
params['c0'].vary = False

params['q0'].value = q_init
params['q0'].vary = False

params['V'].vary = False

**Fitting the model**

Now we can use the `fit()` method and print out the fit report. The first function parameter is the data, the second is the params object defined above, and the third is the independent variable.

In [8]:
lmfit_results = c_model.fit(concentration,params,t=time)
print(lmfit_results.fit_report())

[[Model]]
    Model(conc_index)
[[Fit Statistics]]
    # function evals   = 81
    # data points      = 51
    # variables        = 3
    chi-square         = 0.039
    reduced chi-square = 0.001
    Akaike info crit   = -359.414
    Bayesian info crit = -353.618
[[Variables]]
    c0:      3.023799 (fixed)
    q0:      1.916201 (fixed)
    k1:      0.00222362 +/- 0.000193 (8.67%) (init= 0.01)
    k2:     -0.00437038 +/- 0.000633 (14.49%) (init= 0.01)
    qmax:    0.18167960 +/- 0.006684 (3.68%) (init= 1)
    index:   0 (fixed)
    V:       670 (fixed)
[[Correlations]] (unreported correlations are <  0.100)
    C(k1, k2)                    = -0.997 
    C(k2, qmax)                  =  0.739 
    C(k1, qmax)                  = -0.705 



**Visual check of goodness of fit**

The `lmfit` function has a convenient `plot()` function that shows the residuals and the fitted line along with the curve generated using the initial values.

In [10]:
lmfit_results.plot()

(<matplotlib.figure.Figure at 0xbaf2b70>,
 <matplotlib.gridspec.GridSpec at 0xbae7eb8>)